# Wild Drone LLM Workshop: Part 1 - Simple LLM Travel Agent

Welcome to Part 1 of the Wild Drone LLM Workshop! In this notebook, you'll learn the fundamentals of Large Language Models (LLMs) and build your first AI agent.

## What You'll Learn

1. **LLM Fundamentals**: How LLMs work and their capabilities
2. **Tool-Calling Agents**: Building agents that can interact with external tools
3. **Agent Architecture**: Understanding system prompts and agent design
4. **Practical Application**: Creating a weather-based travel planning agent

## Workshop Structure

- **Setup and Installation**: Configure your environment and API keys
- **LLM Basics**: Understand how language models work
- **Tool Calling**: Learn to give LLMs "superpowers" with external functions
- **Travel Agent**: Build a complete travel planning assistant
- **Experimentation**: Modify prompts and tools to see how agents behave

Let's get started!

## Setup and Installation

Before we begin, let's install the required libraries and set up our environment.

### API Key Setup (Required!)

You need a **Gemini API key** to run this workshop. Get one free from: https://aistudio.google.com/app/apikey

**Recommended Setup:**
1. Copy `.env.example` to `.env`
2. Add your API key: `GEMINI_API_KEY=your_actual_key_here`
3. The notebook will automatically load it securely

**Alternative:** Set environment variable in your terminal:
```bash
export GEMINI_API_KEY=your_actual_key_here
```

**Security Note:** Never commit API keys to version control! The `.env` file is already in `.gitignore`.

In [1]:
# Install required packages
!pip install litellm matplotlib numpy python-dotenv

# Note: If using OpenAI instead of Gemini, also install:
# !pip install openai

Defaulting to user installation because normal site-packages is not writeable


In [21]:
# Configuration and Setup
import os
import litellm
from dotenv import load_dotenv
from llm_agents import add_parameters_schema, TravelAgent

# Load environment variables from .env file (if it exists)
load_dotenv()

# MODEL CONFIGURATION
# Get Gemini API key from environment variable
gemini_api_key = os.getenv('GEMINI_API_KEY')
# Set the API key for litellm
os.environ['GEMINI_API_KEY'] = gemini_api_key
# Set the model to use throughout the workshop
model_name = "gemini/gemini-2.5-flash"  # Using stable model

# Understanding LLMs and Simple Tool-Calling Agents

## How to use LLM APIs
We can interact through LLMs from different providers (OpenAI, Gemini, Antrhophic) and get completions to our prompts. Try modifying the prompt, max tokens or temperature in the following code block

In [31]:
system_prompt = "You are a helpful cooking assistant. Answer in a very concise manner."
user_prompt = "What ingredients do I need to make a spanish tortilla?"

response = litellm.completion(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=1000,  # Shorter responses
        temperature=0.1, # Lower temperature for more factual responses
    )
print(response.choices[0].message.content)

Potatoes, eggs, onion (optional), olive oil, salt.


## Tool Calling: Giving LLMs Superpowers

While LLMs are great at generating text, they have limitations:
- No access to real-time information
- Can't perform calculations reliably
- Can't interact with external systems

In [ ]:
# Let's create a travel agent with no tools
weather_uninformed_travel_agent = TravelAgent(model=model_name)

# Set an empty list of tools (no tools)
weather_uninformed_travel_agent.tools = []

# Define a system prompt for the travel agent
weather_uninformed_travel_agent.system_prompt = """
You are a helpful travel planning assistant.
Your task is to help users decide between travel destinations.
Answer in a friendly and concise manner (max 2-3 sentences).
"""

# Test the uninformed travel agent
question = "I want to travel to Paris or Tokyo next month. Which destination should I choose?"

print("🤖 Testing our Weather-Uninformed Travel Agent:")
print("-" * 50)
response = weather_uninformed_travel_agent.chat(question)

🤖 Testing our Weather-Uninformed Travel Agent:
--------------------------------------------------
- User Question: I want to travel to Paris or Tokyo next month. Which destination should I choose?
- Available tools: []
- Final Answer (no tools used): For a romantic escape filled with art, history, and charming cafes, Paris is an excellent choice. If you're seeking a vibrant, futuristic city with unique culture and incredible culinary experiences, Tokyo awaits!
- Final Answer (no tools used): For a romantic escape filled with art, history, and charming cafes, Paris is an excellent choice. If you're seeking a vibrant, futuristic city with unique culture and incredible culinary experiences, Tokyo awaits!




**Tool calling** solves this by allowing LLMs to call external functions/APIs!

### How Tool Calling Works:
1. **Define Tools**: Specify what functions the LLM can call
2. **LLM Decides**: The model chooses which tool to use based on context
3. **Execute Tool**: The function is called with LLM-provided parameters
4. **Return Results**: Tool output is sent back to the LLM
5. **Generate Response**: LLM uses tool results to create final answer

Let's build a weather-based travel agent as an example!

In [42]:
# STUDENTS: MODIFY THESE TOOLS AND PROMPTS!
# This is where you can see and change how the agent works

# Tool: Weather Forecast
@add_parameters_schema(
    city={
        "type": "string", 
        "description": "Name of the city to get weather for"
    },
    days={
        "type": "integer", 
        "description": "Number of days to forecast (1-7)", 
        "minimum": 1, 
        "maximum": 7
    }
)
def get_weather_forecast(city: str, days: int = 3) -> dict:
    """Get weather forecast for a specific city and number of days"""
    import random
    
    conditions = ["Sunny", "Partly Cloudy", "Cloudy", "Rainy", "Stormy"]
    
    forecast = []
    for day in range(1, days + 1):
        forecast.append({
            "day": day,
            "condition": random.choice(conditions),
            "temperature": random.randint(10, 30)
        })

    # Build output string
    forecast_str = f"Weather forecast for {city}:\n"
    for day_forecast in forecast:
        forecast_str += f" Day {day_forecast['day']}: {day_forecast['condition']}, {day_forecast['temperature']}°C\n"

    return forecast_str

In [43]:
# Test the tool: weather forecast for Paris for 3 days
weather_forecast = get_weather_forecast("Paris", 3)
print(weather_forecast)

Weather forecast for Paris:
 Day 1: Rainy, 14°C
 Day 2: Rainy, 25°C
 Day 3: Partly Cloudy, 20°C



In [57]:
# Now let's create and test a travel agent that is weather-informed
weather_informed_travel_agent = TravelAgent(model=model_name)

# Connect the tools defined in the notebook
weather_informed_travel_agent.tools = [get_weather_forecast]

# Set the system prompt defined in the notebook
weather_informed_travel_agent.system_prompt = """
You are a helpful travel planning assistant.
Your task is to help users decide between travel destinations.
Use the tools provided to make an informed recommendation.
Answer in a friendly and concise manner (max 2-3 sentences).
"""
# Test our travel agent
question = """I'm hesitating between flying to Paris or Rome for a 3-day trip next weekend."""

print("🤖 Testing our Weather-Informed Travel Agent:")
print("-" * 50)

# Get a response from the travel agent
response = weather_informed_travel_agent.chat(question)

🤖 Testing our Weather-Informed Travel Agent:
--------------------------------------------------
- User Question: I'm hesitating between flying to Paris or Rome for a 3-day trip next weekend.
- Available tools: ['get_weather_forecast']
- Tools called: ['get_weather_forecast', 'get_weather_forecast']
- Executing get_weather_forecast with args: {'days': 3, 'city': 'Paris'}
- Tool result: Weather forecast for Paris:
 Day 1: Sunny, 15°C
 Day 2: Partly Cloudy, 11°C
 Day 3: Rainy, 30°C

- Executing get_weather_forecast with args: {'city': 'Rome', 'days': 3}
- Tool result: Weather forecast for Rome:
 Day 1: Partly Cloudy, 28°C
 Day 2: Cloudy, 18°C
 Day 3: Cloudy, 28°C

- Final Answer: For a 3-day trip next weekend, Rome looks like a great choice with partly cloudy to cloudy skies and warm temperatures ranging from 18°C to 28°C. Paris has more varied weather, including a rainy day with a high of 30°C.


# 🚀 Advanced Exercise: Build a Complete Travel Recommendation Agent

Now let's enhance our agent with transportation options! We'll add tools to check flight and train prices, then build an agent that balances cost and sustainability in its recommendations.

## Your Mission

You have been provided with **complete, working tools** for flight and train prices. Your tasks are:

1. **Study the tools**: Understand how they work and what data they provide
2. **Create a system prompt**: Write a prompt that balances cost and sustainability  
3. **Connect the tools**: Add both tools to create an enhanced travel agent
4. **Test and iterate**: Try different prompts to see how they affect recommendations

The tools include realistic pricing, CO2 emissions data, and booking time effects. Your job is to create an agent that uses this data wisely!

In [ ]:
# Complete Flight Prices Tool (Ready to Use!)
@add_parameters_schema(
    origin={
        "type": "string", 
        "description": "Departure city"
    },
    destination={
        "type": "string", 
        "description": "Destination city"
    },
    days={
        "type": "integer", 
        "description": "How many days ahead to search", 
        "minimum": 1, 
        "maximum": 30
    }
)
def get_flight_prices(origin: str, destination: str, days: int = 7) -> str:
    """Get flight prices between two cities with CO2 emissions data"""
    import random
    
    airlines = ["BudgetAir", "SkyConnect", "EuroWings", "FlightLine", "AirEurope"]
    
    # Base price calculation (affected by advance booking)
    base_price = random.randint(80, 400)
    
    # Price adjustments based on days ahead
    if days <= 3:
        price_multiplier = 1.5  # Last minute booking
    elif days <= 7:
        price_multiplier = 1.2
    elif days >= 21:
        price_multiplier = 0.8  # Early booking discount
    else:
        price_multiplier = 1.0
    
    final_price = int(base_price * price_multiplier) + random.randint(-30, 50)
    airline = random.choice(airlines)
    
    # CO2 emissions (flights emit ~90-120g CO2 per km per passenger)
    # Rough distance estimation for European cities
    distance_km = random.randint(400, 2000)
    co2_per_km = random.randint(90, 120)
    co2_emissions = int(distance_km * co2_per_km / 1000)  # kg CO2
    
    flight_duration = distance_km // 600 + random.randint(1, 2)  # hours
    
    return f"Flight {origin} → {destination}: {airline} €{final_price} ({flight_duration}h flight, CO2: {co2_emissions}kg)"

# Test the flight tool
print("Testing complete flight tool:")
print(get_flight_prices("Paris", "Rome", 7))

Testing complete flight tool:
✈️ Flight Paris → Rome: AirEurope €480 (1h flight, CO2: 52kg)


In [ ]:
# Complete Train Prices Tool (Ready to Use!)
@add_parameters_schema(
    origin={
        "type": "string", 
        "description": "Departure city"
    },
    destination={
        "type": "string", 
        "description": "Destination city"
    },
    days={
        "type": "integer", 
        "description": "How many days ahead to search", 
        "minimum": 1, 
        "maximum": 30
    }
)
def get_train_prices(origin: str, destination: str, days: int = 7) -> str:
    """Get train prices between two cities with environmental benefits"""
    import random
    
    # Some routes might not be available
    unavailable_routes = [
        ("London", "Rome"), ("Madrid", "Berlin"), ("Stockholm", "Athens")
    ]
    
    route_check = (origin, destination)
    reverse_route = (destination, origin)
    
    if route_check in unavailable_routes or reverse_route in unavailable_routes:
        return f"🚫 No direct train route available from {origin} to {destination}"
    
    train_operators = ["EuroRail Express", "HighSpeed Connect", "Regional Railways", "TransEurope"]
    
    # Base price calculation (usually 40-70% of flight prices)
    base_price = random.randint(40, 250)
    
    # Price adjustments for advance booking
    if days <= 3:
        price_multiplier = 1.3
    elif days >= 14:
        price_multiplier = 0.7  # Better advance discounts than flights
    else:
        price_multiplier = 1.0
    
    final_price = int(base_price * price_multiplier) + random.randint(-20, 30)
    operator = random.choice(train_operators)
    
    # Journey time (2-12 hours depending on distance and train type)
    journey_time = random.randint(3, 10)
    
    # CO2 emissions (trains emit ~14-20g CO2 per km per passenger - much lower!)
    distance_km = random.randint(400, 1500)
    co2_per_km = random.randint(14, 20)
    co2_emissions = int(distance_km * co2_per_km / 1000)  # kg CO2
    
    return f"Train {origin} → {destination}: {operator} €{final_price} ({journey_time}h journey, CO2: {co2_emissions}kg)"

# Test the train tool  
print("Testing complete train tool:")
print(get_train_prices("Paris", "Rome", 7))

Testing complete train tool:
🚆 Train Paris → Rome: HighSpeed Connect €226 (3h journey, CO2: 18kg)


## 🎯 Student Task: Create Your Enhanced Travel Agent

Now it's your turn! You have two complete tools ready to use:
- `get_flight_prices()` - includes pricing, duration, and CO2 emissions
- `get_train_prices()` - includes pricing, journey time, and much lower CO2 emissions

### Your Tasks:

1. **Write a System Prompt**: Create a prompt that guides your agent to balance cost and sustainability
2. **Connect the Tools**: Add both transportation tools to your agent
3. **Test Your Agent**: Try different scenarios to see how it recommends

### System Prompt Guidelines:

Your prompt should tell the agent:
- Its role and personality 
- How to weigh cost vs environmental impact
- What information to prioritize
- How to structure recommendations

### Example Priorities to Consider:

- **Eco-Warrior**: "Always recommend the most sustainable option"
- **Budget-Conscious**: "Find the cheapest option that's reasonably sustainable"  
- **Balanced**: "Consider both cost and environmental impact equally"
- **Time-Sensitive**: "Factor in journey time as well as cost and emissions"

In [ ]:
# TODO: STUDENT TASK - Create and configure your enhanced agent
# 1. Create a new TravelAgent instance
enhanced_travel_agent = TravelAgent(model=model_name)

# 2. Add the weather, flight and trains tool to the agent

# 3. Write your enhanced system prompt here to consider cost and sustainability
enhanced_travel_agent.system_prompt = """
Write your system prompt here!

Consider including:
- Your agent's role and personality
- How to balance cost vs environmental impact  
- What factors to prioritize (price, CO2, time, comfort)
- How to structure recommendations
- Tone (friendly, professional, expert, etc.)
"""

## 🧪 Test Your Enhanced Travel Agent

Once you've created your enhanced agent, test it with different scenarios! Try the test questions below to see how well your agent balances cost and sustainability.

### Test Scenarios to Try:

1. **"I want to travel from London to Barcelona next weekend. I care about both cost and environmental impact. What do you recommend?"**

2. **"I'm planning a trip from Paris to Amsterdam. I'm on a tight budget but also care about the environment."**

3. **"Compare travel options from Madrid to Rome - I want the fastest option regardless of cost."**

4. **"I want to visit Berlin from Vienna, and I have plenty of time. What's the most sustainable option?"**

5. **"Find me the best weekend trip options from Munich - weather is important to me."**

### Things to Notice:

- How does your agent weigh cost vs CO2 emissions?
- Does it explain the trade-offs clearly?
- How does it factor in journey time?
- Does it check weather as expected?

In [ ]:
# Test your enhanced travel agent here!
# Make sure you've completed the agent creation above first

test_question = """
I want to travel from London to Barcelona next weekend. 
I care about both cost and environmental impact. 
What do you recommend?
"""

enhanced_travel_agent.chat(test_question)